In [9]:
!pip install ultralytics opencv-python numpy PyYAML

In [10]:
import sqlite3, os, cv2, numpy as np
from pathlib import Path
from PIL import Image
import base64

def extract_rgb_frames(bag_dir, out_dir, topic_hint='color', max_frames=50):
    bag_path = Path(bag_dir)
    db3_files = list(bag_path.glob('*.db3'))
    if not db3_files:
        raise FileNotFoundError(f"No .db3 found under {bag_dir}")
    conn = sqlite3.connect(str(db3_files[0]))
    c = conn.cursor()

    # list all topics
    topics = [row[0] for row in c.execute('SELECT name FROM topics')]
    print("Topics:", topics)

    # pick the first one containing 'image'
    topic = [t for t in topics if 'image' in t and topic_hint in t]
    if not topic:
        topic = [t for t in topics if 'image' in t]
    topic = topic[0]
    print("Using topic:", topic)

    # get topic_id
    tid = c.execute("SELECT id FROM topics WHERE name=?", (topic,)).fetchone()[0]
    msgs = c.execute("SELECT data FROM messages WHERE topic_id=?", (tid,)).fetchmany(max_frames)

    os.makedirs(out_dir, exist_ok=True)
    for i,(data,) in enumerate(msgs):
        # raw image bytes are serialized as sensor_msgs/Image in ROS2 (binary blob)
        # We'll use OpenCV imdecode assuming JPEG encoding
        # Find JPEG start marker (FF D8)
        idx = data.find(b'\xff\xd8')
        if idx >= 0:
            img_bytes = data[idx:]
            img = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), cv2.IMREAD_COLOR)
            if img is not None:
                out = Path(out_dir)/f"frame_{i:03d}.jpg"
                cv2.imwrite(str(out), img)
    conn.close()
    print(f"[OK] Saved frames to {out_dir}")


In [11]:
extract_rgb_frames(
    bag_dir="/content/drive/MyDrive/Perceptra/Challenge Surveys/office/rosbag2_2025_10_20-16_09_39",
    out_dir="/content/drive/MyDrive/Perceptra/frames/office",
    max_frames=30
)

extract_rgb_frames(
    bag_dir="/content/drive/MyDrive/Perceptra/Challenge Surveys/bathroom/rosbag2_2025_10_20-16_47_22",
    out_dir="/content/drive/MyDrive/Perceptra/frames/bathroom",
    max_frames=30
)


Topics: ['/imu', '/odom', '/livox/imu', '/livox/lidar', '/zed/zed_node/depth/depth_registered/compressedDepth', '/zed/zed_node/odom', '/scan', '/tf', '/zed/zed_node/rgb/image_rect_color/compressed', '/tf_static', '/zed/zed_node/rgb/camera_info']
Using topic: /zed/zed_node/rgb/image_rect_color/compressed
[OK] Saved frames to /content/drive/MyDrive/Perceptra/frames/office
Topics: ['/imu', '/odom', '/livox/imu', '/livox/lidar', '/scan', '/tf', '/tf_static', '/zed/zed_node/depth/depth_registered/compressedDepth', '/zed/zed_node/odom', '/zed/zed_node/rgb/image_rect_color/compressed', '/zed/zed_node/rgb/camera_info']
Using topic: /zed/zed_node/rgb/image_rect_color/compressed
[OK] Saved frames to /content/drive/MyDrive/Perceptra/frames/bathroom


In [12]:
# Sanity Check
!yolo detect predict model=yolov8l.pt source="/content/drive/MyDrive/Perceptra/frames/office" conf=0.4 save=True


Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLOv8l summary (fused): 112 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs

image 1/30 /content/drive/MyDrive/Perceptra/frames/office/frame_000.jpg: 384x640 1 chair, 1 potted plant, 3 tvs, 2772.9ms
image 2/30 /content/drive/MyDrive/Perceptra/frames/office/frame_001.jpg: 384x640 1 chair, 1 potted plant, 3 tvs, 3016.8ms
image 3/30 /content/drive/MyDrive/Perceptra/frames/office/frame_002.jpg: 384x640 1 chair, 1 potted plant, 3 tvs, 2041.6ms
image 4/30 /content/drive/MyDrive/Perceptra/frames/office/frame_003.jpg: 384x640 1 chair, 1 potted plant, 3 tvs, 1993.7ms
image 5/30 /content/drive/MyDrive/Perceptra/frames/office/frame_004.jpg: 384x640 1 chair, 2 potted plants, 3 tvs, 1997.3ms
image 6/30 /content/drive/MyDrive/Perceptra/frames/office/frame_005.jpg: 384x640 1 chair, 1 potted plant, 3 tvs, 2320.6ms
image 7/30 /content/drive/MyDrive/Perceptra/frames/office/frame_006.jpg: 384x640 1 chair, 1 po

In [13]:
!python /content/drive/MyDrive/Perceptra/scripts/detect_and_overlay.py \
  --room-yaml "/content/drive/MyDrive/Perceptra/Challenge Surveys/office/room.yaml" \
  --room-pgm  "/content/drive/MyDrive/Perceptra/Challenge Surveys/office/room.pgm" \
  --rgb-dir   "/content/drive/MyDrive/Perceptra/frames/office" \
  --out-dir   results/office \
  --model yolov8n.pt \
  --only-best-frame



[INFO] Using best frame frame_019.jpg
[OK] Saved 2 detections → results/office/detections.json
[OK] Saved overlay → results/office/map_with_detections.png


In [14]:
!python /content/drive/MyDrive/Perceptra/scripts/detect_and_overlay.py \
  --room-yaml "/content/drive/MyDrive/Perceptra/Challenge Surveys/bathroom/room.yaml" \
  --room-pgm  "/content/drive/MyDrive/Perceptra/Challenge Surveys/bathroom/room.pgm" \
  --rgb-dir   "/content/drive/MyDrive/Perceptra/frames/bathroom" \
  --out-dir   results/bathroom \
  --model yolov8n.pt \
  --only-best-frame


[OK] Saved 0 detections → results/bathroom/detections.json
[OK] Saved overlay → results/bathroom/map_with_detections.png
